In [1]:
# import packets
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import matplotlib.pyplot as plt
import numpy as np

In [2]:
# hyperparas
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
LR=5e-3

cuda


In [3]:
# define augmentation strategies

def gauss_noise_tensor(img):
    assert isinstance(img, torch.Tensor)
    dtype = img.dtype
    if not img.is_floating_point():
        img = img.to(torch.float32)
    
    sigma = 25.0
    
    out = img + sigma * torch.randn_like(img)
    
    if out.dtype != dtype:
        out = out.to(dtype)
        
    return out

group_a=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

group_b=transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

group_c=transforms.Compose([
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)), 
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

group_d=transforms.Compose([
    transforms.ColorJitter(brightness=.5, hue=.3), 
    transforms.ToTensor(),
    gauss_noise_tensor,
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

group_e=transforms.Compose([
    transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10), 
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_names=['A', 'B', 'C', 'D', 'E']
test_groups=[group_a, group_b, group_c, group_d, group_e]

In [4]:
# prepare data

def prepareData(group_i):
  T=test_groups[group_i]
  trainset=torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=T)
  testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=test_groups[0])
  
  #trainset=torch.utils.data.Subset(trainset, range(int(len(trainset)/40)))
  #testset=torch.utils.data.Subset(testset, range(int(len(testset)/20)))
  
  trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=4)
  testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=4)
  return trainloader, testloader

In [5]:
# model structure

class SE(nn.Module):
    '''Squeeze-and-Excitation block.'''

    def __init__(self, in_planes, se_planes):
        super(SE, self).__init__()
        self.se1 = nn.Conv2d(in_planes, se_planes, kernel_size=1, bias=True)
        self.se2 = nn.Conv2d(se_planes, in_planes, kernel_size=1, bias=True)

    def forward(self, x):
        out = F.adaptive_avg_pool2d(x, (1, 1))
        out = F.relu(self.se1(out))
        out = self.se2(out).sigmoid()
        out = x * out
        return out


class Block(nn.Module):
    def __init__(self, w_in, w_out, stride, group_width, bottleneck_ratio, se_ratio):
        super(Block, self).__init__()
        # 1x1
        w_b = int(round(w_out * bottleneck_ratio))
        self.conv1 = nn.Conv2d(w_in, w_b, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(w_b)
        # 3x3
        num_groups = w_b // group_width
        self.conv2 = nn.Conv2d(w_b, w_b, kernel_size=3,
                               stride=stride, padding=1, groups=num_groups, bias=False)
        self.bn2 = nn.BatchNorm2d(w_b)
        # se
        self.with_se = se_ratio > 0
        if self.with_se:
            w_se = int(round(w_in * se_ratio))
            self.se = SE(w_b, w_se)
        # 1x1
        self.conv3 = nn.Conv2d(w_b, w_out, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(w_out)

        self.shortcut = nn.Sequential()
        if stride != 1 or w_in != w_out:
            self.shortcut = nn.Sequential(
                nn.Conv2d(w_in, w_out,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(w_out)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        if self.with_se:
            out = self.se(out)
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class RegNet(nn.Module):
    def __init__(self, cfg, num_classes=10):
        super(RegNet, self).__init__()
        self.cfg = cfg
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(0)
        self.layer2 = self._make_layer(1)
        self.layer3 = self._make_layer(2)
        self.layer4 = self._make_layer(3)
        self.linear = nn.Linear(self.cfg['widths'][-1], num_classes)

    def _make_layer(self, idx):
        depth = self.cfg['depths'][idx]
        width = self.cfg['widths'][idx]
        stride = self.cfg['strides'][idx]
        group_width = self.cfg['group_width']
        bottleneck_ratio = self.cfg['bottleneck_ratio']
        se_ratio = self.cfg['se_ratio']

        layers = []
        for i in range(depth):
            s = stride if i == 0 else 1
            layers.append(Block(self.in_planes, width,
                                s, group_width, bottleneck_ratio, se_ratio))
            self.in_planes = width
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def RegNetX_200MF():
    cfg = {
        'depths': [1, 1, 4, 7],
        'widths': [24, 56, 152, 368],
        'strides': [1, 1, 2, 2],
        'group_width': 8,
        'bottleneck_ratio': 1,
        'se_ratio': 0,
    }
    return RegNet(cfg)


In [6]:
# get model instance

model=RegNetX_200MF().to(device)
print(model)
summary(model, (3, 32, 32))

RegNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Block(
      (conv1): Conv2d(64, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3, bias=False)
      (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (layer2): Sequential(

In [7]:
# optimization paradigm

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [8]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx%20==0:
          print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                      % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    sum_loss = 0
    sum_acc = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            sum_loss+=(test_loss/(batch_idx+1))
            sum_acc+=(100.*correct/total)
    #test_losses.append(sum_loss/len(testloader))
    test_accs.append(sum_acc/len(testloader))
    print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (sum_loss/len(testloader), sum_acc/len(testloader), correct, total))

In [9]:
start_epoch=0
avg_best_accs=[]
for i in range(1, len(test_groups)):
  trainloader, testloader=prepareData(i)
  #model=RegNetX_200MF().to(device)
  test_accs=[]
  print('======= start test group:', i, '=======')
  for epoch in range(start_epoch, start_epoch+40):
      train(epoch)
      test(epoch)
      scheduler.step()
  test_accs.sort(reverse=True)
  avg_best_accs.append(sum(test_accs[:5])/5.00)

Files already downloaded and verified
Files already downloaded and verified
======= start test group: 1 =======

Epoch: 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0 391 Loss: 3.212 | Acc: 10.938% (14/128)
20 391 Loss: 2.521 | Acc: 12.054% (324/2688)
40 391 Loss: 2.390 | Acc: 15.053% (790/5248)
60 391 Loss: 2.296 | Acc: 17.290% (1350/7808)
80 391 Loss: 2.225 | Acc: 18.962% (1966/10368)
100 391 Loss: 2.156 | Acc: 20.653% (2670/12928)
120 391 Loss: 2.111 | Acc: 22.036% (3413/15488)
140 391 Loss: 2.078 | Acc: 23.105% (4170/18048)
160 391 Loss: 2.043 | Acc: 24.233% (4994/20608)
180 391 Loss: 2.010 | Acc: 25.207% (5840/23168)
200 391 Loss: 1.979 | Acc: 26.318% (6771/25728)
220 391 Loss: 1.953 | Acc: 27.255% (7710/28288)
240 391 Loss: 1.926 | Acc: 28.106% (8670/30848)
260 391 Loss: 1.904 | Acc: 28.975% (9680/33408)
280 391 Loss: 1.888 | Acc: 29.682% (10676/35968)
300 391 Loss: 1.866 | Acc: 30.458% (11735/38528)
320 391 Loss: 1.848 | Acc: 31.194% (12817/41088)
340 391 Loss: 1.828 | Acc: 31.862% (13907/43648)
360 391 Loss: 1.815 | Acc: 32.395% (14969/46208)
380 391 Loss: 1.798 | Acc: 33.011% (16099/48768)
99 100 Loss: 1.549 | Acc: 45.262% (4493/10000)

E

In [10]:
print(avg_best_accs)

[87.80170226399171, 73.19278701708485, 10.867333203964202, 91.36902079859581]


In [11]:
print(list(zip(test_names, avg_best_accs)))

[('A', 87.80170226399171), ('B', 73.19278701708485), ('C', 10.867333203964202), ('D', 91.36902079859581)]
